# **Agent #5: The Logistician**
**Overview**

The Logistician handles the scheduling, booking, and coordination so you can focus on the event itself. It will perform actions like:
- checking databases
- enforcing budgets
- handling scheduling errors autonomously

The Agent is part of **Squad 2: The Engineering Squad (Execution & Reliability)** for the **Living Lab** Platform.

## 1. **Install Dependencies**
We need to install dspy-ai for the agent framework and the Google Gemini SDK.

In [54]:
# %pip install dspy-ai google-generativeai

In [70]:
import dspy
import os
import random
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
# Configure Gemini (using the model from your original notebook)
# If running in Colab, make sure you set your API key in the secrets or environment
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = input("Enter Google API Key: ")

gemini = dspy.LM('gemini/gemini-2.5-flash', api_key=os.environ["GOOGLE_API_KEY"])
dspy.configure(lm=gemini)

## 2. **The Mock Database**
Agents interact with the world through data. Since we are in a notebook, we will simulate a real database using Python dictionaries.
- **Calendar:** Simulates available and busy slots.
- **Budget:** Simulates project funds.

In [87]:
# --- MOCK DATA ---

# 1. Calendar Data
# Note: Oct 20th has busy slots to test the agent's logic.
calendar_db = {
    "2023-10-20": ["10:00", "11:00"], 
    "2023-10-21": [], 
    "2023-10-22": []
}

# 2. Financial Data
project_budget = {"remaining": 5000}

print("✅ Mock Database Loaded.")

✅ Mock Database Loaded.


## 3. **Creating Agent Tools**
In DSPy, tools are Python functions. The agent reads the docstring (description) to understand when and how to use them.

We will implement four tools:

- **check_budget:** Enforces financial constraints.
- **check_calendar_availability:** Reads the database.
- **book_venue:** Writes to the database (includes failure simulation).
- **send_email_invitation:** Simulates communication.

In [88]:
def check_budget(estimated_cost: str) -> str:
    """
    Checks if there is enough budget. Input must be a numeric string (e.g., "3000").
    """
    try:
        cost = int(estimated_cost)
        remaining = project_budget["remaining"]
        if cost > remaining:
            return f"Budget Rejected: Insufficient funds. Cost {cost} exceeds remaining {remaining}."
        return f"Budget Approved: Cost {cost} is within limit. Remaining: {remaining}."
    except ValueError:
        return "Error: Invalid cost format. Please provide a numeric string."

def check_calendar_availability(date: str) -> str:
    """
    Checks if a specific date (YYYY-MM-DD) is available.
    """
    if date in calendar_db:
        busy_slots = calendar_db[date]
        if not busy_slots:
            return f"Date {date} is completely free."
        return f"Date {date} has busy slots at: {', '.join(busy_slots)}."
    return f"Date {date} is completely free."

def book_venue(date: str, time: str) -> str:
    """
    Books the venue for a specific date and time (Format: YYYY-MM-DD, HH:MM).
    """
    # # SIMULATION: Random System Failure (20% chance)
    # # This tests the agent's ability to retry/recover.
    # if random.random() < 0.2: 
    #     return "Error: Venue is unexpectedly under maintenance. Booking failed."
    
    if date in calendar_db and time in calendar_db[date]:
        return f"Error: Slot {time} on {date} is already booked."
    
    if date not in calendar_db:
        calendar_db[date] = []
    calendar_db[date].append(time)
    
    return f"Success: Venue booked for {date} at {time}."

def send_email_invitation(recipient_list: str, subject: str) -> str:
    """
    Sends email invitations to a list of recipients.
    """
    return f"Success: Sent invitations to {recipient_list} with subject '{subject}'."

# Group tools into a list for the agent
my_tools = [check_budget, check_calendar_availability, book_venue, send_email_invitation]

## 4. **Defining the Agent (DSPy Signature)**
We define the agent's behavior using a Signature. This acts as the "System Prompt."
- Objectives: We explicitly list the KRs so the model knows its priorities (Execution, Resilience, Efficiency).
- Module: We use dspy.ReAct to bind the Signature to the Tools.

In [89]:
class LogisticianSignature(dspy.Signature):
    """
    You are Agent #5: The Logistician.
    GOAL: Orchestrate logistics operations autonomously.
    
    OBJECTIVES (Key Results):
    - KR1 (Execution): Execute tool chains (Budget -> Venue -> Invite) with 99.9% success.
    - KR2 (Resilience): Automatically recover from 'Booking Failures' (Busy/Error) by selecting valid alternatives.
    - KR3 (Efficiency): Reduce administrative overhead.
    
    CRITICAL RULES:
    1. Check Budget (check_budget) FIRST. If budget fails, Stop.
    2. ALWAYS check availability (check_calendar_availability) before booking.
    3. If a booking fails (e.g., Room Full/Maintenance), DO NOT GIVE UP. 
    4. AUTO-CORRECTION: IF a slot is busy, SILENTLY increment the time by 1 hour (e.g. 10:00 -> 11:00) and BOOK AGAIN immediately. Repeat until you find a free slot. DO NOT ask the user for permission.
    """
    question = dspy.InputField(desc="The logistics request from the user.")
    answer = dspy.OutputField(desc="Final confirmation of actions taken (e.g., 'Booked 12:00').")

# Initialize the Agent
# We increase 'max_iters' to give it enough "thinking steps" to retry multiple times
agent = dspy.ReAct(LogisticianSignature, tools=my_tools, max_iters=10)

print("✅ Agent Initialized.")

✅ Agent Initialized.


## 5. **Validation Scenarios**
We will now run two test cases to verify the agent meets the System Requirements (Key Results).

**Test 1: Complex Chain (Execution & Efficiency)**

This test validates **KR1 (Execution)** and **KR3 (Efficiency)**.
- **Input:** A complex request requiring three distinct actions: Checking Budget, Booking a Venue, and Sending an Invitation.
- **Expected Behavior:** The agent should execute all three tools in the correct logical order within a single interaction loop.

In [90]:
print("🚀 TEST CASE 1: KR1 (Execution) & KR3 (Efficiency)")

# Scenario: Budget Check -> Calendar Check -> Booking -> Email Invite.
question_kr1 = "Set up 'Hybrid Event' on 2023-10-21 at 09:00, cost 3000. Send email to 'student@tu.ac.th'."

# Execute Agent
response_kr1 = agent(question=question_kr1)

print(f"\nRequest: {question_kr1}")
print(f"Result:  {response_kr1.answer}")

🚀 TEST CASE 1: KR1 (Execution) & KR3 (Efficiency)

Request: Set up 'Hybrid Event' on 2023-10-21 at 09:00, cost 3000. Send email to 'student@tu.ac.th'.
Result:  Booked 09:00, email sent to student@tu.ac.th.


**Test 2: Conflict Resolution (Resilience)**

This test validates KR2 (Resilience).

- **Input:** A request for 10:00 on Oct 20th. We know from our Mock DB that this slot is busy.
- **Expected Behavior:** The agent should detect the conflict, reason that it needs an alternative, and successfully book a different slot (e.g., 12:00) autonomously.

In [92]:
print("🚀 TEST CASE 2: KR2 (Resilience) - Conflict Recovery")

# Scenario: User asks for 10:00 (Busy). Agent MUST find next slot.
question_kr2 = "I need to book 'Workshop A' on 2023-10-20 at 10:00 with cost 1000."

# Execute Agent
response_kr2 = agent(question=question_kr2)

print(f"\nRequest: {question_kr2}")
print(f"Result:  {response_kr2.answer}")

🚀 TEST CASE 2: KR2 (Resilience) - Conflict Recovery

Request: I need to book 'Workshop A' on 2023-10-20 at 10:00 with cost 1000.
Result:  Booked 2023-10-20 at 13:00.


In [93]:
print("🔍 EXPLORING EDGE CASES\n")

# Example 3: Budget Constraint Test (Critical Rule #1)
# Scenario: The user asks for an event that is too expensive.
# Expected Behavior: The agent should check the budget, see it fails, and STOP immediately.
question3 = "I want to host a Luxury Gala on 2023-10-22. Estimated cost is 8000."
response3 = agent(question=question3)

print(f"Q: {question3}")
print(f"A: {response3.answer}")
print("-" * 50)

# Example 4: Pure Information Retrieval
# Scenario: The user just wants to know availability, not book.
# Expected Behavior: The agent should act as a search engine, calling 'check_calendar' only.
question4 = "Can you just check if 2023-10-20 is free? Don't book anything yet."
response4 = agent(question=question4)

print(f"Q: {question4}")
print(f"A: {response4.answer}")
print("-" * 50)

# Example 5: Successful Booking on a Free Day
# Scenario: A standard request on a day we know is empty (Oct 22).
# Expected Behavior: Smooth execution (Budget -> Book -> Invite).
question5 = "Book a Team Sync on 2023-10-22 at 14:00, cost 200. Invite 'manager@corp.com'."
response5 = agent(question=question5)

print(f"Q: {question5}")
print(f"A: {response5.answer}")

🔍 EXPLORING EDGE CASES

Q: I want to host a Luxury Gala on 2023-10-22. Estimated cost is 8000.
A: Budget rejected: Insufficient funds. Cannot proceed with the Luxury Gala.
--------------------------------------------------
Q: Can you just check if 2023-10-20 is free? Don't book anything yet.
A: 2023-10-20 has busy slots at: 10:00, 11:00, 12:00, 13:00.
--------------------------------------------------
Q: Book a Team Sync on 2023-10-22 at 14:00, cost 200. Invite 'manager@corp.com'.
A: Booked 14:00


In [86]:
# We call .inspect_history(n=1) to see the "Reasoning Trace".
gemini.inspect_history(n=1)





[2026-01-27T23:16:45.130681]

System message:

Your input fields are:
1. `question` (str): The logistics request from the user.
2. `trajectory` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Final confirmation of actions taken (e.g., 'Booked 12:00').
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are Agent #5: The Logistician.
        GOAL: Orchestrate logistics operations autonomously.
        
        OBJECTIVES (Key Results):
        - KR1 (Execution): Execute tool chains (Budget -> Venue -> Invite) with 99.9% success.
        - KR2 (Resilience): Automatically recover from 'Booking Failures' (Busy/Error) by selecting valid alternatives.
        - KR3 (Efficiency): Reduce administrative ov